In [1]:
import pandas as pd
import numpy as np
import os
import time
import datetime

provinces = ['Bangkok','Chanthaburi','Kanchanaburi','Songkhla','Chiang Mai','Khon Kaen']
left_join = dict(how='left', left_index=True, right_index=True)
parse_dates = dict(parse_dates=True, index_col=0)

In [2]:
test_data = {}
fire = pd.read_csv("fire_2016_to_2020_localize.csv", **parse_dates)

for p in provinces:
    ## Read PM data
    df = pd.read_csv(f"../../{p}/Test/{p}.csv", **parse_dates)
    df.rename(columns={"PM2.5(µg/m3)":"PM2.5"}, inplace=True)

    ## Read Temp,Wind data
    wind = pd.read_csv(f"../../{p}/Test/3H_wind_{p}.csv", **parse_dates)[['WindDir','Wind Speed(km/h)']]
    temp = pd.read_csv(f"../../{p}/Test/3H_temperature_{p}.csv", **parse_dates)[['Temp(C)']]


    # Merge
    df = df.merge(wind, **left_join).merge(temp, **left_join).merge(fire, **left_join)

    # Wind & Temp fill
    df = df.ffill(limit=3).bfill(limit=3)
    print(f"Missing = {len(pd.date_range(df.index[0],df.index[-1],freq='H')) - len(df) }")

    test_data[p] = df

Missing = 2044
Missing = 2044
Missing = 2044
Missing = 1941
Missing = 2046
Missing = 2055


In [39]:
for p in provinces:
    test_data[p].to_csv(f"new test/{p}_new_test.csv")

In [3]:
test_data['Bangkok']

,PM2.5,WindDir,Wind Speed(km/h),Temp(C),Cambodia_frp,Myanmar_frp,Thailand_frp,Lao_PDR_frp
Time,,,,,,,,
2019-03-18 08:00:00,45.6,120.0,14.0,33.4,0.000000,0.0000,0.000000,0.000000
2019-03-18 09:00:00,46.6,120.0,14.0,33.4,0.000000,0.0000,0.000000,0.000000
2019-03-18 10:00:00,47.7,120.0,14.0,33.4,0.000000,0.0000,0.000000,0.000000
2019-03-18 11:00:00,43.7,120.0,14.0,33.4,0.000000,0.0000,0.000000,0.000000
2019-03-18 12:00:00,41.6,120.0,14.0,33.4,10.406856,25.2634,6.844051,26.476113
...,...,...,...,...,...,...,...,...
2020-03-18 18:00:00,24.9,210.0,7.0,33.0,0.000000,0.0000,0.000000,0.000000
2020-03-18 19:00:00,27.0,260.0,6.0,30.4,0.000000,0.0000,0.000000,0.000000
2020-03-18 20:00:00,26.1,260.0,6.0,30.4,0.000000,0.0000,0.000000,0.000000


In [9]:
idx= pd.date_range('2019-03-18 08:00:00','2020-03-15 22:00:00', freq='H')

In [10]:
idx[idx.hour.isin([6,12,18,0])]

DatetimeIndex(['2019-03-18 12:00:00', '2019-03-18 18:00:00',
               '2019-03-19 00:00:00', '2019-03-19 06:00:00',
               '2019-03-19 12:00:00', '2019-03-19 18:00:00',
               '2019-03-20 00:00:00', '2019-03-20 06:00:00',
               '2019-03-20 12:00:00', '2019-03-20 18:00:00',
               ...
               '2020-03-13 12:00:00', '2020-03-13 18:00:00',
               '2020-03-14 00:00:00', '2020-03-14 06:00:00',
               '2020-03-14 12:00:00', '2020-03-14 18:00:00',
               '2020-03-15 00:00:00', '2020-03-15 06:00:00',
               '2020-03-15 12:00:00', '2020-03-15 18:00:00'],
              dtype='datetime64[ns]', length=1454, freq=None)

In [13]:
have = test_data['Bangkok'].iloc[:-72].index
have[have.hour.isin([6,12,18,0])]

DatetimeIndex(['2019-03-18 12:00:00', '2019-03-18 18:00:00',
               '2019-03-19 00:00:00', '2019-03-19 06:00:00',
               '2019-03-19 12:00:00', '2019-03-19 18:00:00',
               '2019-03-20 00:00:00', '2019-03-20 06:00:00',
               '2019-03-20 12:00:00', '2019-03-20 18:00:00',
               ...
               '2020-03-13 12:00:00', '2020-03-13 18:00:00',
               '2020-03-14 00:00:00', '2020-03-14 06:00:00',
               '2020-03-14 12:00:00', '2020-03-14 18:00:00',
               '2020-03-15 00:00:00', '2020-03-15 06:00:00',
               '2020-03-15 12:00:00', '2020-03-15 18:00:00'],
              dtype='datetime64[ns]', name='Time', length=1112, freq=None)

In [14]:
1454-1112

342

In [15]:
have[have.hour.isin([6,12,18,0])].pop(0)

AttributeError: 'DatetimeIndex' object has no attribute 'pop'